In [1]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'YOLOv3'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install pytorch_lightning

In [3]:
pip install albumentations==0.4.6

In [13]:
import numpy as np
import torch
import torch.nn as nn
from model import darknet
from data import dataset
import util
import cv2
from PIL import ImageShow


%load_ext autoreload
%autoreload 2
torch.cuda.is_available()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [6]:
device = 'cuda'
CONFIG_FILE = 'yolov3.cfg'
CONFIG_PATH = '/content/drive/My Drive/YOLOv3/cfg/{}'.format(CONFIG_FILE)
darknet.parse_cfg(CONFIG_PATH)

[{'angle': '0',
  'batch': '64',
  'burn_in': '1000',
  'channels': '3',
  'decay': '0.0005',
  'exposure': '1.5',
  'height': '608',
  'hue': '.1',
  'learning_rate': '0.001',
  'max_batches': '500200',
  'momentum': '0.9',
  'policy': 'steps',
  'saturation': '1.5',
  'scales': '.1,.1',
  'steps': '400000,450000',
  'subdivisions': '16',
  'type': 'net',
  'width': '608'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '32',
  'pad': '1',
  'size': '3',
  'stride': '1',
  'type': 'convolutional'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '64',
  'pad': '1',
  'size': '3',
  'stride': '2',
  'type': 'convolutional'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '32',
  'pad': '1',
  'size': '1',
  'stride': '1',
  'type': 'convolutional'},
 {'activation': 'leaky',
  'batch_normalize': '1',
  'filters': '64',
  'pad': '1',
  'size': '3',
  'stride': '1',
  'type': 'convolutional'},
 {'activation': 'linear', 'from': '-3', 'type'

In [7]:
blocks = darknet.parse_cfg(CONFIG_PATH)
darknet.create_modules(blocks)

[{'type': 'net', 'batch': '64', 'subdivisions': '16', 'width': '608', 'height': '608', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '32', 'size': '3', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '2', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '32', 'size': '1', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'shortcut', 'from': '-3', 'activation': 'linear'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '128', 'siz

({'angle': '0',
  'batch': '64',
  'burn_in': '1000',
  'channels': '3',
  'decay': '0.0005',
  'exposure': '1.5',
  'height': '608',
  'hue': '.1',
  'learning_rate': '0.001',
  'max_batches': '500200',
  'momentum': '0.9',
  'policy': 'steps',
  'saturation': '1.5',
  'scales': '.1,.1',
  'steps': '400000,450000',
  'subdivisions': '16',
  'type': 'net',
  'width': '608'},
 ModuleList(
   (0): Sequential(
     (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (Activation_0): LeakyReLU(negative_slope=0.1, inplace=True)
   )
   (1): Sequential(
     (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
     (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (Activation_1): LeakyReLU(negative_slope=0.1, inplace=True)
   )
   (2): Sequential(
     (conv_2): Conv2d(

In [15]:
WEIGHTS_FILE = 'yolov3.weights'
WEIGHTS_PATH = '/content/drive/My Drive/YOLOv3/weights/{}'.format(WEIGHTS_FILE)
model = darknet.Darknet(CONFIG_PATH)
model.load_weights(WEIGHTS_PATH)
model.cuda()
input = darknet.get_test_input()
pred = model(input, device)
result = util.write_result(pred, 0.5, 80)
print(result)
print(result.size())

[{'type': 'net', 'batch': '64', 'subdivisions': '16', 'width': '608', 'height': '608', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '32', 'size': '3', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '2', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '32', 'size': '1', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'shortcut', 'from': '-3', 'activation': 'linear'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '128', 'siz

In [16]:
images , labels = dataset.prepare_data()
print(images)
print(labels[1])

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7F5155196A90>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x426 at 0x7F5155479610>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x428 at 0x7F5155479E90>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x425 at 0x7F5155196350>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=481x640 at 0x7F51552CC290>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x478 at 0x7F5155297150>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=381x500 at 0x7F5155232F90>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x488 at 0x7F5155232B50>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x640 at 0x7F515523BD90>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x426 at 0x7F515523B3D0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=427x640 at 0x7F51552445D0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=565x640 at 0x7F5155244F50>, <PI

In [30]:
images = np.array(list(map(lambda x: np.array(x), images)), dtype=object)
display(images[126].shape)
labels[126]

(375, 500, 3)

['62 0.42217 0.716493 0.42062 0.529893',
 '39 0.18036 0.82772 0.09312 0.308133',
 '73 0.72236 0.389267 0.0116 0.126267',
 '73 0.85443 0.394213 0.09602 0.168427',
 '73 0.67057 0.39276 0.01246 0.128667',
 '73 0.68045 0.39272 0.01022 0.130347',
 '73 0.69228 0.395107 0.01284 0.126',
 '73 0.69397 0.759547 0.0587 0.152',
 '73 0.70329 0.39012 0.01758 0.130427',
 '73 0.73517 0.39176 0.1441 0.13056',
 '73 0.70365 0.946067 0.24438 0.085413',
 '77 0.43418 0.07944 0.09648 0.141707',
 '45 0.3492 0.22412 0.08104 0.063547',
 '64 0.23286 0.97588 0.07216 0.04728',
 '73 0.22994 0.248627 0.10952 0.014053',
 '73 0.22808 0.260147 0.11112 0.014053',
 '73 0.23099 0.273973 0.10746 0.01264',
 '77 0.81076 0.16444 0.07972 0.14712',
 '73 0.23152 0.286933 0.10744 0.013867',
 '39 0.43892 0.215467 0.05436 0.176373',
 '']

In [34]:
WEIGHTS_FILE = 'yolov3.weights'
WEIGHTS_PATH = '/content/drive/My Drive/YOLOv3/weights/{}'.format(WEIGHTS_FILE)
model = darknet.Darknet(CONFIG_PATH)
model.load_weights(WEIGHTS_PATH)
model.cuda()
input = darknet.get_test_input(image=images[123]) 
pred = model(input, device)
result = util.write_result(pred, 0.5, 80)
print(result)
print(result.size())

[{'type': 'net', 'batch': '64', 'subdivisions': '16', 'width': '608', 'height': '608', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '32', 'size': '3', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '2', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '32', 'size': '1', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'shortcut', 'from': '-3', 'activation': 'linear'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '128', 'siz